In [1]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits as pyfits
import sys, os
import healpy as hp

sys.path.append('/Users/mcrnogor/Desktop/Swift-GBM/scripts/') # path to the functions script
from calc_BAT_ul import *

from copy import copy
from kapteyn import maputils
import ephem

/Users/mcrnogor/opt/anaconda3/lib/python3.7/site-packages/pyfits/__init__.py:22: PyFITSDeprecationWarning: PyFITS is deprecated, please use astropy.io.fits
  PyFITSDeprecationWarning)  # noqa


In [2]:
events = ["GW200112_155838"]

In [8]:
for x in events:
    
    name = x # looping through the event names 
    
    bat_info = "/Users/mcrnogor/Desktop/Swift-GBM/results_O3b_2020/" + name + "/info.txt"
    lines = open(bat_info).readlines()
    bat_ra = float(lines[25].split()[-1])
    bat_dec = float(lines[26].split()[-1])
    earth_ra = float(lines[23].split()[-1])
    earth_dec = float(lines[24].split()[-1])
    earth_rad = 69. # Earth radius for Swift BAT
    t0 = float(lines[8].split()[-1]) # LIGO trigger time 
    
    # loading the 1-s BAT ligtcurve
    lc_path = "/Users/mcrnogor/Desktop/Swift-GBM/results_O3b_2020/" +name + "/1s_rate_sort.lc"
    lc_1s = pyfits.open(lc_path)
    counts_1s = lc_1s[1].data['COUNTS']
    time_1s = lc_1s[1].data['TIME'] - t0
    rate_1s = counts_1s 

    # starting the light curve from -1 to 30 to match GBM search window
    index = [i for i,x in enumerate(time_1s) if x>=-1 and x<=30]
    time_new = time_1s[index]
    counts_new = counts_1s[index]
    rate_new = rate_1s[index]
    std = np.std(rate_new) # std from the 0.5 second lightcurve
    
    Ndets_tot = 32768.0 # total dets
    Ndets_active = float(lines[36].split()[-1]) # current number of active dets
    Ndet_ratio = Ndets_active / Ndets_tot

    IDs = [1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 29, 30, 31, 32, 33]
    rate_std = std # the std of rates from the LC
    rate_upper_limit = 5*rate_std
    # print("5sigma rate upper limit: ", rate_upper_limit)
    
    ul_5sigma = [] 
    
    for i in IDs:
        grid_id = i 
        # print("grid_id: ", grid_id)

        # using energy bin 15-350 and ignoring 350-500
        chan_low = 0 
        chan_hi = 3

        # getting the NITRATES DRM table
        drm_tab = get_drm_tab(grid_id)

        # response matrix using selected energy bins and corrected for number of active dets
        drm_matrix = drm_tab['MATRIX'][:,chan_low:(chan_hi+1)] * Ndet_ratio 

        # find the flux that gives an expected rate equal to the rate upper limit
        flux_upper_limit = rate2band_eflux(rate_upper_limit, drm_matrix,\
                                       drm_tab['ENERG_LO'], drm_tab['ENERG_HI'],\
                                       alpha, beta, Epeak, flux_elo, flux_ehi)
        #print("5-sigma flux upper limit [erg/cm2/s]: ", flux_upper_limit)
        ul_5sigma.append(flux_upper_limit)
        np.save(os.path.join('/Users/mcrnogor/Desktop/Swift-GBM/results_O3b_2020/', name + "_BAT_gridID_ul.npy"), ul_5sigma)
        
    ul_min = ul_5sigma[12]
    ul_max = ul_5sigma[15]
    
    # plotting the BAT FoV
    bat_img = "/Users/mcrnogor/Desktop/Swift-GBM/results_O3b_2020/" + name + "/batfov.fits"
    res = .25
    nra = int(360. / res)
    ndec = int(180. / res)

    header = {'NAXIS'  : 2, 'NAXIS1': nra, 'NAXIS2': ndec,
    'CTYPE1' : 'RA---MOL',
    'CRVAL1' : bat_ra, 'CRPIX1' : int(.5 * nra), 'CUNIT1' : 'deg', 'CDELT1' : -0.25,
    'CTYPE2' : 'DEC--MOL',
    'CRVAL2' : bat_dec, 'CRPIX2' : int(.5 * ndec), 'CUNIT2' : 'deg', 'CDELT2' : 0.25,
          }

    fits = maputils.FITSimage(bat_img, hdunr=0).reproject_to(header)
    
    # producing a healpy map
    nside = 512 # as GBM
    npix = hp.nside2npix(nside)
    val = np.zeros(npix, np.float64)
    th, ph = hp.pix2ang(nside, np.arange(npix))
    ra, dec = np.degrees([ph, .5 * np.pi - th])

    i, j = fits.proj.topixel((ra, dec))
    val = fits.dat[j.astype(int), i.astype(int)]

    vmax = val[~np.isnan(val)].max()
    vmin = val[~np.isnan(val)].min()
    k = (ul_max - ul_min)/(vmax - vmin)
    m = ul_max - k*vmax
    val = k*val+m # rescale FOV map as an upper limit map
    val[np.isnan(val)] = hp.UNSEEN
    val[np.isnan(val)] = hp.UNSEEN
    # print("Min: %.3e" % val[val > hp.UNSEEN].min())
    # print("Max: %.3e" % val[val > hp.UNSEEN].max())
    
    Earth_limb_region_x = []
    Earth_limb_region_y = []
    
    # mask out the Earth
    earth_vec = hp.ang2vec(np.radians(90. - earth_dec), np.radians(earth_ra))
    earth_pix = hp.query_disc(nside, earth_vec, np.radians(earth_rad))
    val[earth_pix] = hp.UNSEEN
    plt.figure()
    hp.mollview(val, rot=0, fig=3, norm='log', cmap='RdPu', title=name)
    hp.graticule()
    plt.annotate(r"0$^o$", (0.02, 0.02), size=15)
    plt.annotate(r"-30$^o$", (0.31, 0.02), horizontalalignment='center', size=15)
    plt.annotate(r"30$^o$", (-0.31, 0.02), horizontalalignment='center', size=15)
    plt.annotate(r"-60$^o$", (0.65, 0.02), horizontalalignment='center', size=15)
    plt.annotate(r"60$^o$", (-0.65, 0.02), horizontalalignment='center', size=15)
    plt.annotate(r"-90$^o$", (0.97, 0.02), horizontalalignment='center', size=15)
    plt.annotate(r"90$^o$", (-0.97, 0.02), horizontalalignment='center', size=15)
    plt.annotate(r"-120$^o$", (1.30, 0.02), horizontalalignment='center', size=15)
    plt.annotate(r"120$^o$", (-1.30, 0.02), horizontalalignment='center', size=15)
    plt.annotate(r"-150$^o$", (1.65, 0.02), horizontalalignment='center', size=15)
    plt.annotate(r"150$^o$", (-1.65, 0.02), horizontalalignment='center', size=15)
    plt.annotate(r"+30$^o$", (-1.97,  0.45), horizontalalignment='center', verticalalignment='center', size=15)
    plt.annotate(r"-30$^o$", (-1.99, -0.45), horizontalalignment='center', verticalalignment='center', size=15)
    plt.annotate(r"+60$^o$", (-1.40,  0.80), horizontalalignment='center', verticalalignment='center', size=15)
    plt.annotate(r"-60$^o$", (-1.40, -0.80), horizontalalignment='center', verticalalignment='center', size=15)
    zlabel = r"5$\sigma$ Flux U.L. [erg/cm2/s]"
    plt.annotate(zlabel, (0.5, 0.05), horizontalalignment='center', size=10, xycoords='figure fraction')
    plt.annotate("15-350 keV", (.85, .25), size=10, xycoords='figure fraction')
    plt.annotate("Normal Spectrum", (.85, .21), size=10, xycoords='figure fraction')
    plt.annotate("1-sec Emission", (.85, .17), size=10, xycoords='figure fraction')

    os.path.join('Check', 'train_set'),
    np.save(os.path.join('/Users/mcrnogor/Desktop/Swift-GBM/results_O3b_2020/', name + ".npy"), val)
    plt.savefig(os.path.join('/Users/mcrnogor/Desktop/Swift-GBM/results_O3b_2020/', name + ".png"), dpi=300)
    plt.clf()
    
    BAT_coord = np.zeros(2)
    BAT_coord[0] = bat_ra
    BAT_coord[1] = bat_dec
    np.save(os.path.join('/Users/mcrnogor/Desktop/Swift-GBM/results_O3b_2020/', name + "_BAT_coord.npy"), BAT_coord)

/Users/mcrnogor/opt/anaconda3/lib/python3.7/site-packages/healpy/projaxes.py:543: UserWarning: 0.0 180.0 -180.0 180.0
  pmin / dtor, pmax / dtor, mmin / dtor, mmax / dtor
/Users/mcrnogor/opt/anaconda3/lib/python3.7/site-packages/healpy/projaxes.py:658: UserWarning: The interval between parallels is 30 deg -0.00'.
  vdeg, varcmin
/Users/mcrnogor/opt/anaconda3/lib/python3.7/site-packages/healpy/projaxes.py:666: UserWarning: The interval between meridians is 30 deg -0.00'.
  vdeg, varcmin


<Figure size 432x288 with 0 Axes>

<Figure size 612x388.8 with 0 Axes>